In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
places = pd.read_pickle('data/places_preprocessed.pkl')
users = pd.read_pickle('data/users_preprocessed.pkl')
accidents = pd.read_pickle('data/accidents_preprocessed.pkl')

In [3]:
master_table = accidents.set_index("accident_id").join(places.set_index("accident_id"))
master_table = master_table.join(users.set_index("accident_id"))

In [4]:
master_table = master_table.reset_index()

We are trying to model type of injury to a person. In each accident there could be multiple pepole - we need to handle that 

In [ ]:
cols = list(master_table.columns)
cols

In [ ]:
char_cols = [
    'lighting',
 'localization',
 'intersection_type',
 'weather',
 'collision_type',
 'road_category',
 'road_regime',
 'reserved_lane',
 'road_gradient',
 'road_plan',
 'road_condition',
 'infrastructure',
 'accident_situation',
 'user_type',
 'injury_type',
 'sex',
 'equipment_used',
 'pedestrian_action',
 'pedestrian_alone'
]

In [ ]:
pieces = []
for col in char_cols:
    tmp_series = master_table[col].value_counts()
    tmp_series.name = col
    pieces.append(tmp_series)
df_value_counts = pd.concat(pieces, axis=1)

In [ ]:
pieces

In [ ]:
master_table.shape

In [ ]:
master_table.injury_type.value_counts()

In [5]:
master_table['y'] = master_table.injury_type.map({"Unscathed": 0, 
                             "Light injury": 0,
                              "Hospitalized wounded": 1,
                              "Killed": 1
                             })

In [ ]:
master_table.y.mean()

Using a stratified train test split. Dependence on particular accident doesn't matter much - only few observations.

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(master_table, master_table.y, stratify=master_table.y, test_size=0.3)

In [ ]:
# x_train.drop('y', axis = 1, inplace = True)
# x_test.drop('y', axis = 1, inplace = True)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

### First model

Logistic regression with 2 features - user type sex and lightning. Also removing nas

In [7]:
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
temp_x_train = x_train[['y', 'sex', 'user_type', 'lighting', 'year']].dropna()

In [11]:
def use_all_cols(df):
    all_columns = " + ".join([i for i in df.columns if i != 'y'])
    my_formula = "y ~ " + all_columns
    return my_formula

In [ ]:
use_all_cols(temp_x_train)

In [ ]:
mod = sm.GLM.from_formula(formula=use_all_cols(temp_x_train), data=temp_x_train, family=sm.families.Binomial())
res = mod.fit()
res.summary()

In [ ]:
roc_auc_score(temp_x_train.y, res.predict())

Drop in AUC on test set not bad. However performance is rather questionable.

In [8]:
import matplotlib.pyplot as plt

In [ ]:
fpr, tpr, _ = roc_curve(temp_x_train.y, res.predict())
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
# plt.show()

### Model 2

In [9]:
temp_x_train = x_train[['y',
                        'lighting',
 'localization',
 'weather',
 'collision_type',
 'year',
 'month',
 'hour',
 'road_condition',
 'user_type',
 'sex',
]].dropna()

In [12]:
mod = sm.GLM.from_formula(formula=use_all_cols(temp_x_train), data=temp_x_train, family=sm.families.Binomial())
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:              1250468
Model:                            GLM   Df Residuals:                  1250433
Model Family:                Binomial   Df Model:                           34
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -6.2267e+05
Date:                Wed, 08 Apr 2020   Deviance:                   1.2453e+06
Time:                        22:04:23   Pearson chi2:                 1.26e+06
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==================================================================================================================================
                                                                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          2.6231      1.290      2.033      0.042       0.094       5.152
lighting[T.Night with public lighting not lit]                     0.1052      0.025      4.216      0.000       0.056       0.154
lighting[T.Night with public lighting on]                          0.1033      0.007     15.024      0.000       0.090       0.117
lighting[T.Night without public lighting]                          0.3963      0.008     50.035      0.000       0.381       0.412
lighting[T.Twilight or dawn]                                       0.1507      0.009     16.163      0.000       0.132       0.169
localization[T.Out of agglomeration]                               1.0605      0.005    203.765      0.000       1.050       1.071
weather[T.Dazzling weather]                                        0.3143      0.024     13.370      0.000       0.268       0.360
weather[T.Fog - smoke]                                             0.0855      0.026      3.246      0.001       0.034       0.137
weather[T.Heavy rain]                                             -0.0691      0.019     -3.607      0.000      -0.107      -0.032
weather[T.Light rain]                                             -0.2325      0.014    -16.329      0.000      -0.260      -0.205
weather[T.Normal]                                                 -0.0884      0.013     -6.907      0.000      -0.113      -0.063
weather[T.Other]                                                   0.0802      0.029      2.766      0.006       0.023       0.137
weather[T.Snow - hail]                                            -0.2050      0.037     -5.610      0.000      -0.277      -0.133
weather[T.Strong wind - storm]                                     0.2101      0.043      4.874      0.000       0.126       0.295
collision_type[T.Three or more vehicles - multiple collisions]    -0.5422      0.011    -50.716      0.000      -0.563      -0.521
collision_type[T.Three vehicles and more - in chain]              -1.4030      0.015    -95.622      0.000      -1.432      -1.374
collision_type[T.Two vehicles - by the side]                      -0.1954      0.006    -30.117      0.000      -0.208      -0.183
collision_type[T.Two vehicles - from the rear]                    -0.6838      0.009    -76.820      0.000      -0.701      -0.666
collision_type[T.Two vehicles - frontal]                           0.3988      0.008     52.856      0.000       0.384       0.414
collision_type[T.Without collision]                                0.4159      0.009     47.901      0.000       0.399       0.433
road_condition[T.flooded]                                          0.0

In [13]:
roc_auc_score(temp_x_train.y, res.predict())

0.7044569516018537